
# Finetuning DeciYolo on a Custom Dataset

Background: Our research team developed a new deep learning architecture that competes with YOLOv8 using our efficient AutoNAC algorithm. 
The new model, named DeciYolo, incorporates quantization-aware RepVGG blocks into the model architecture to ensure compatibility with Post-Training Quantization, making it more flexible and usable for different hardware configurations.

In this tutorial, we will demonstrate how to fine-tune the small variant of the DeciYolo, DeciYoloS on a custom dataset: the [RF Soccer Players Dataset](https://universe.roboflow.com/growingkomab-gmail-com/soccer_players/dataset/14).


0. Installations and dataset setup.
1. Initialize train and validation dataloaders.
2. Instantiate a pre-trained DeciYoloS model.
3. Define training hyperparameters and launch training.
4. Run prediction on a real world input.

## Step 0: Dataset Setup

First, follow this link will download the RF Soccer Players Dataset:
 - Follow the [dataset download page](https://universe.roboflow.com/growingkomab-gmail-com/soccer_players/dataset/14).
 - Sign up or login to Roboflow.
 - Click on download, then select "COCO" as the format. Click "continue".
 - Under "Jupyter" option, you will see the code below, replace "YOUR_API_KEY" with the one given to you in the pop up box.

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="HHHAIeIk8YN2rZ6tP8nR")
project = rf.workspace("growingkomab-gmail-com").project("soccer_players")
dataset = project.version(14).download("coco")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 2.4.5
    Uninstalling pyparsing-2.4.5:
      Successfully uninstalled pyparsing-2.4.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
super-gradients 3.0.8+master requires pyparsing==2.4.5, but you have pyparsing 2.4.7 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to soccer_players-14 in coco:: 100%|██████████| 659/659 [00:00<00:00, 1834.48it/s]


# New Section

In [2]:
! pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113 &> /dev/null
! pip install git+https://github.com/Deci-AI/super-gradients.git@feature/SG-736_deci_yolo_rf100

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Deci-AI/super-gradients.git (to revision feature/SG-736_deci_yolo_rf100) to /tmp/pip-req-build-7xzztun0
  Running command git clone --filter=blob:none --quiet https://github.com/Deci-AI/super-gradients.git /tmp/pip-req-build-7xzztun0
  Running command git checkout -b feature/SG-736_deci_yolo_rf100 --track origin/feature/SG-736_deci_yolo_rf100
  Switched to a new branch 'feature/SG-736_deci_yolo_rf100'
  Branch 'feature/SG-736_deci_yolo_rf100' set up to track remote branch 'feature/SG-736_deci_yolo_rf100' from 'origin'.
  Resolved https://github.com/Deci-AI/super-gradients.git to commit fb9c82218f36522069759bab646ae77957306a2c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pyparsing-2.4.5-py2.py3-none-any.whl (67 kB)
  Attempting uninstall: pyparsing
    Found

## Step 1: Initialize train and validation dataloaders.

In [3]:
from super_gradients.training.datasets.detection_datasets.coco_format_detection import COCOFormattedDetectionDataset
from super_gradients.training.transforms.transforms import DetectionMosaic, DetectionRandomAffine, DetectionHSV, DetectionHorizontalFlip, DetectionPaddedRescale, DetectionStandardize, DetectionTargetsFormatTransform
from super_gradients.training.utils.utils import load_func
from super_gradients.training.utils.detection_utils import DetectionCollateFN
from super_gradients.training import dataloaders
from super_gradients.training.datasets.datasets_utils import worker_init_reset_seed

train_transforms = [DetectionMosaic(prob=1., input_dim=(640,640)),
                    DetectionRandomAffine(degrees=0., scales=(0.5,1.5), shear=0., target_size=(640, 640), filter_box_candidates=False, border_value=128),
                    DetectionHSV(prob=1., hgain=5, vgain=30, sgain=30),
                    DetectionHorizontalFlip(prob=0.5),
                    DetectionPaddedRescale(input_dim=(640, 640), max_targets=300),
                    DetectionStandardize(max_value=255),
                    DetectionTargetsFormatTransform(max_targets=300, input_dim=(640, 640), output_format="LABEL_CXCYWH")
                    ]
trainset = COCOFormattedDetectionDataset(data_dir="/content/soccer-players-2", images_dir="train", json_annotation_file="train/_annotations.coco.json", input_dim=(640, 640), ignore_empty_annotations=False, transforms=train_transforms)

val_transforms = [DetectionPaddedRescale(input_dim=(640, 640), max_targets=300), DetectionStandardize(max_value=255) ,DetectionTargetsFormatTransform(max_targets=300, input_dim=(640, 640), output_format="LABEL_CXCYWH")]

valset = COCOFormattedDetectionDataset(data_dir="/content/soccer-players-2", images_dir="valid", json_annotation_file="valid/_annotations.coco.json", input_dim=(640, 640), ignore_empty_annotations=False, transforms=val_transforms)

train_loader = dataloaders.get(dataset=trainset,
                               dataloader_params={"shuffle": True,
                                                  "batch_size": 16,
                                                  "min_samples": 512,
                                                  "num_workers": 2,
                                                  "drop_last": False,
                                                  "pin_memory": True,
                                                  "worker_init_fn": worker_init_reset_seed,
                                                  "collate_fn":DetectionCollateFN()})

valid_loader = dataloaders.get(dataset=valset,
                               dataloader_params={"shuffle": False,
                                                  "batch_size": 32,
                                                  "num_workers": 2,
                                                  "drop_last": False,
                                                  "pin_memory": True,
                                                  "collate_fn":DetectionCollateFN()})



[2023-04-13 13:19:00] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


[2023-04-13 13:19:04] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-04-13 13:19:04] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-04-13 13:19:04] WARNING - export.py - Failed to import pytorch_quantization
[2023-04-13 13:19:04] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-04-13 13:19:04] INFO - env_sanity_check.py - Library check is not supported when super_gradients installed through "git+https://github.com/..." command
Caching annotations: 100%|██████████| 33/33 [00:00<00:00, 1085.26it/s]
[2023-04-13 13:19:04] INFO - dataloaders.py - Using min_samples=512


In [4]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
train_params = {
  "warmup_mode": "linear_epoch_step",
  "warmup_initial_lr":  1e-6,
  "lr_warmup_epochs": 3,
  "initial_lr":  5e-4,
  "lr_mode": "cosine",
  "cosine_final_lr_ratio": 0.1,
  "optimizer": "AdamW",
  "optimizer_params":{"weight_decay": 0.00001},
  "zero_weight_decay_on_bias_and_bn": True,
  "lr_warmup_epochs": 3,
  "warmup_mode": "linear_epoch_step",
  "initial_lr": 5e-4,
  "optimizer_params":{"weight_decay": 0.0001},
  "ema": True,
  "ema_params":{"decay": 0.9, "decay_type": "threshold"},
  "max_epochs": 100,
  "mixed_precision": True,
  "loss": PPYoloELoss(use_static_assigner=False, num_classes=3, reg_max=16),
  "valid_metrics_list":[DetectionMetrics_050(score_thres=0.1, top_k_predictions=300, num_cls=3, normalize_targets=True,
                                             post_prediction_callback=PPYoloEPostPredictionCallback(score_threshold=0.01, nms_top_k=1000, max_predictions=300, nms_threshold=0.7))],

  "metric_to_watch": 'mAP@0.50'}

In [ ]:
from super_gradients.training import Trainer
from super_gradients.common.object_names import Models
from super_gradients.training import models

trainer = Trainer(experiment_name="deci_yolo_s_soccer_players", ckpt_root_dir="/content/sg_checkpoints_dir/")
net = models.get(Models.DECIYOLO_S, num_classes=3, pretrained_weights="coco")
trainer.train(model=net, training_params=train_params, train_loader=train_loader, valid_loader=valid_loader)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


[2023-04-13 13:19:06] WARNING - sg_trainer.py - Train dataset size % batch_size != 0 and drop_last=False, this might result in smaller last batch.
[2023-04-13 13:19:12] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}
[2023-04-13 13:19:17] INFO - utils.py - NumExpr defaulting to 2 threads.


The console stream is now moved to /content/sg_checkpoints_dir/deci_yolo_s_soccer_players/console_Apr13_13_19_17.txt


[2023-04-13 13:19:26] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               1          (1 available on the machine)
    - Dataset size:                 114        [Samples]
    - Dataloader size:              32         [Batches]
    - Batch size per GPU:           16         (batch_size)
    - Batch Accumulate:             1          (batch_accumulate)
    - Total batch size:             16         (num_gpus * batch_size)
    - Effective Batch size:         16         (num_gpus * batch_size * batch_accumulate)
    - Iterations per epoch:         7          (len(train_set) / total_batch_size)
    - Gradient updates per epoch:   7          (len(train_set) / effective_batch_size)

[2023-04-13 13:19:26] INFO - sg_trainer.py - Started training for 100 epochs (0/99)

Train epoch 0:   0%|          | 0/32 [00:00<?, ?it/s]